In [1]:
#Data Download Path：     https://hershey.dbi.udel.edu/textmining/export/
#site    iTextMine: integrated text-mining system for large-scale knowledge extraction from the literature

In [3]:
# 预训练模型GLoVE 
import json
import pandas as pd
import numpy as np
df_pmc = pd.read_json('rlims.pmc.json',lines=True)
'''
设置实体类型标签
B_Protein蛋白质标签
B_Trigger触发器标签
'''
tag2id = {
    'O':1,
    'B_Protein':2,
    'B_Site':3,
    'B_SiteOther':4,
    'B_Trigger':5
}

In [4]:
df_pmc.head(1)

,_id,article_type,docId,entity,fig_id,fig_lable,id,parent,pmcid,pmid,relation,sec_type,sentence,table_id,table_lable,text,title_offset,type,xml_sec_type
0,{'$oid': '5bb1845bb5e292a71605ac5c'},research-article,PMC3062564-36,"{'RL:k78d': {'duid': 'RL:k78d', 'attribute': [...",NaN,NaN,36,"[22.0, 35.0]",PMC3062564,21445358,{'RL:tN3x': {'relationType': 'PHOSPHORYLATION'...,results,"[{'charStart': 0, 'charEnd': 139, 'index': 0},...",NaN,NaN,To further confirm the role of c-Src in PDGF-i...,NaN,P,NaN


In [3]:
# for item in df_pmc.itertuples():
#     entity = item.entity
#     sentences = item.sentence
#     text = item.text
#     for i in entity:
#         a = entity[i]
#         entityType = a['entityType']
#         entityText = a['entityText']
#         print(entityText)
#         print(entityType)
#     break

In [4]:
# text_type

In [5]:
import keras
from keras.preprocessing import text as T
vectorizer = T.Tokenizer(lower=False,split=
' ',num_words=None,char_level=False,filters=',.')
#对文本进行词典解析
document_after = df_pmc['text'].values
vectorizer.fit_on_texts(document_after)
text_sequences = vectorizer.texts_to_sequences(document_after)
#id词对应表
word_index = vectorizer.word_index
index_word = vectorizer.index_word
word_index[' '] = 0
index_word[0] = ' '
#词典数目
word_vec = len(index_word)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
from keras.preprocessing import sequence
#设置input_length为2000
max_len = 2000
#对数据进行补齐操作
text_sequences_padded = sequence.pad_sequences(text_sequences, maxlen=max_len)

In [7]:
shape = text_sequences_padded.shape

In [9]:

def set_type(_type):
    if _type=='Protein':
        value = 2
    elif _type=='Trigger':
        value = 5
    elif _type=='SiteOther':
        value = 4
    elif _type=='Site':
        value = 3
    else:
        value = 1
    return value
#对数据中文本的每个单词进行进行标注用作训练数据
y = np.ones((text_sequences_padded.shape[0],2000))
for i,line in enumerate(text_sequences_padded):
    entitys = df_pmc.loc[i,'entity']
    text_type = {}
    for entity in entitys:
        a = entitys[entity]
        entityType = a['entityType']
        entityText = a['entityText']
        text_type[entityText] = entityType
    for j,key in enumerate(list(line)):
        if index_word[key] in text_type.keys():
            type_value = text_type[index_word[key]]
            value = set_type(type_value)
            y[i,j] = value
y = y.astype(np.int)

In [10]:
#建立简单的bilstm+crf模型
max_features = word_vec
embedding_dims = 200
sentence_length = 2000
NUM_CLASS = 5
DROPOUT_RATE = 0.5
from keras.models import *
from keras.layers import *
from keras_contrib.layers.crf import CRF
model = Sequential()
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=sentence_length))
model.add(Bidirectional(LSTM(20,return_sequences=True),merge_mode='sum'))
model.add(Dropout(DROPOUT_RATE))
model.add(TimeDistributed(Dense(NUM_CLASS)))
model.add(Dropout(DROPOUT_RATE))
#NUM_CLASS为标签数目
crf_layer = CRF(NUM_CLASS)
model.add(crf_layer)
model.compile('rmsprop', loss=crf_layer.loss_function, metrics=[crf_layer.accuracy])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/anaconda3/lib/python3.6/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/anaconda3/lib/python3.6/site-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2000, 200)         126808200 
_________________________________________________________________
bidirectional_1 (Bidirection (None, 2000, 20)          35360     
_________________________________________________________________
dropout_1 (Dropout)          (None, 2000, 20)          0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 2000, 5)           105       
_________________________________________________________________
dropout_2 (Dropout)          (None, 2000, 5)           0         
_________________________________________________________________
crf_1 (CRF)                  (None, 2000, 5)           65        
Total params: 126,843,730
Trainable params: 126,843,730
Non-trainable params: 0
______________________________________________________________

In [11]:
#对输出进行Embedding操作
def Get_Embendding(y):
    results = np.zeros((y.shape[0],y.shape[1],5))
    for i,line in enumerate(y):
        result = np.zeros((y.shape[1],5))
        for j,key in enumerate(list(line)):
            result[j,key-1] = 1
        results[i] = result
    return results
y_embendding = Get_Embendding(y)

In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(text_sequences_padded,y_embendding,test_size=0.3,random_state=1)

In [ ]:
model.fit(x_train,y_train,batch_size=200,epochs=3)

Instructions for updating:
Use tf.cast instead.


/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:107: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 126808200 elements. This may consume a large amount of memory.
  num_elements)


Epoch 1/3
 92800/116035 [======================>.......] - ETA: 31:53 - loss: 0.1495 - crf_viterbi_accuracy: 0.9387

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-451bf0c20155>", line 1, in <module>
    model.fit(x_train,y_train,batch_size=200,epochs=3)
  File "/anaconda3/lib/python3.6/site-packages/keras/engine/training.py", line 1039, in fit
    validation_steps=validation_steps)
  File "/anaconda3/lib/python3.6/site-packages/keras/engine/training_arrays.py", line 199, in fit_loop
    outs = f(ins_batch)
  File "/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py", line 2715, in __call__
    return self._call(inputs)
  File "/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py", line 2675, in _call
    fetched = self._callable_fn(*array_vals)
  File "/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1439, in __call__
    run_metadata_ptr)
KeyboardInter

KeyboardInterrupt: 

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/zmq/eventloop/zmqstream.py", line 432, in _run_callback
    callback(*args, **kwargs)
  File "/anaconda3/lib/python3.6/site-packages/tornado/stack_context.py", line 276, in null_wrapper
    return fn(*args, **kwargs)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 636, in _abort_queues
    self._abort_queue(stream)
  File "/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py", line 661, in _abort_queue
    pol

In [1]:
model.evaluate(x_test,y_test)

NameError: name 'model' is not defined

In [2]:
#简易的测试
prediction = model.predict_classes(x_test[:2])

NameError: name 'model' is not defined

In [ ]:
np.argmax([i for i in prediction[1]])

In [ ]:
index_word[x_test[1][1891]]

In [ ]:
' '.join([index_word[i] for i in x_test[1]])

In [280]:
model.save('model.h5')